In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import re

# Load pre-trained bangla-bert-base tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
model = AutoModel.from_pretrained("sagorsarker/bangla-bert-base")

# Define Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)

        self.dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, v, k, q, mask):
        batch_size = q.size(0)

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = scaled_attention.permute(0, 2, 1, 3)  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = scaled_attention.reshape(batch_size, -1, self.d_model)  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

# Define Scaled Dot-Product Attention
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = torch.matmul(q, k.transpose(-2, -1))  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = torch.tensor(k.size(-1)).type(torch.float32)  # Corrected line
    scaled_attention_logits = matmul_qk / torch.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = torch.nn.functional.softmax(scaled_attention_logits, dim=-1)  # (..., seq_len_q, seq_len_k)
    output = torch.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

# Define Transformer Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output) # Remove training argument
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output) # Remove training argument
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

# Define Point Wise Feed Forward Network
def point_wise_feed_forward_network(d_model, dff):
    return nn.Sequential(
        nn.Linear(d_model, dff),
        nn.ReLU(),
        nn.Linear(dff, d_model)
    )

# Define Transformer Encoder
class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                                self.d_model)
        self.token_type_embeddings = nn.Embedding(2, d_model) # Embedding for token type IDs

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]

        self.dropout = nn.Dropout(rate)

    def forward(self, x, token_type_ids, training, mask):
        seq_len = x.size(1)

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= torch.math.sqrt(torch.tensor(self.d_model).type_as(x))
        x += self.pos_encoding[:, :seq_len, :]

        # Add token type embeddings
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        x += token_type_embeddings

        x = self.dropout(x)  # Remove training argument

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

# Define Positional Encoding
def positional_encoding(position, d_model):
    angle_rads = get_angles(torch.arange(position)[:, torch.newaxis],
                          torch.arange(d_model)[torch.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = torch.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = torch.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[torch.newaxis, ...]

    return pos_encoding.to(torch.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / torch.pow(10000, (2 * (i//2)) / d_model)
    return pos * angle_rates

# Get sentence embeddings
def get_sentence_embeddings(sentences):
    """
    Generates sentence embeddings for a list of sentences using bangla-bert-base.

    Args:
      sentences: A list of sentences.

    Returns:
      A list of sentence embeddings (tensors).
    """
    sentence_embeddings = []
    for sentence in sentences:
        # print(sentence)
        inputs = tokenizer(sentence, return_tensors='pt')
        outputs = model(**inputs)
        # Use the [CLS] token embedding as the sentence embedding
        sentence_embedding = outputs.last_hidden_state[:, 0, :]
        # sentence_embedding = outputs.last_hidden_state.mean(dim=1)
        # print(sentence_embedding)
        sentence_embeddings.append(sentence_embedding)
    # print(sentence_embeddings)
    return sentence_embeddings

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Lowercase
    return text


# Calculate similarity (updated with Transformer encoder)
def calculate_similarity(text1, text2):
    """
    Calculates the cosine similarity between two texts using
    Transformer encoder on sentence embeddings.

    Args:
      text1: The first text.
      text2: The second text.

    Returns:
      The cosine similarity score.
    """
    # Split texts into sentences
    sentences1 = text1.split('। ')
    sentences2 = text2.split('। ')
    # sentences1 = [preprocess_text(sentence) for sentence in text1.split('। ')]
    # sentences2 = [preprocess_text(sentence) for sentence in text2.split('। ')]

    # Get sentence embeddings
    embeddings1 = get_sentence_embeddings(sentences1)
    embeddings2 = get_sentence_embeddings(sentences2)

    embeddings1 = torch.cat(embeddings1, dim=0)
    embeddings2 = torch.cat(embeddings2, dim=0)

    mean_embeddings1 = embeddings1.mean(dim=0)
    mean_embeddings2 = embeddings2.mean(dim=0)

    similarity = torch.nn.functional.cosine_similarity(mean_embeddings1, mean_embeddings2, dim=0)
    mean_similarity = similarity.mean()
    print(mean_similarity.item())


    # --- Tokenization and Input Preparation ---
    # input_ids1 = []  # List to store token IDs for text1
    # for sentence in sentences1:
    #     inputs = tokenizer(sentence, return_tensors='pt')
    #     input_ids1.extend(inputs['input_ids'].squeeze().tolist())

    # input_ids2 = []  # List to store token IDs for text2
    # for sentence in sentences2:
    #     inputs = tokenizer(sentence, return_tensors='pt')
    #     input_ids2.extend(inputs['input_ids'].squeeze().tolist())

    input_ids1, token_type_ids1 = [], []
    for i, sentence in enumerate(sentences1):
        inputs = tokenizer(sentence, return_tensors='pt')
        input_ids1.extend(inputs['input_ids'].squeeze().tolist())
        token_type_ids1.extend([i % 2] * len(inputs['input_ids'].squeeze().tolist()))  # Alternate 0 and 1

    input_ids2, token_type_ids2 = [], []
    for i, sentence in enumerate(sentences2):
        inputs = tokenizer(sentence, return_tensors='pt')
        input_ids2.extend(inputs['input_ids'].squeeze().tolist())
        token_type_ids2.extend([i % 2] * len(inputs['input_ids'].squeeze().tolist()))  # Alternate 0 and 1


    # Pad the sequences to the same length
    max_length = max(len(input_ids1), len(input_ids2))
    input_ids1 = input_ids1 + [0] * (max_length - len(input_ids1))  # Pad with zeros
    input_ids2 = input_ids2 + [0] * (max_length - len(input_ids2))  # Pad with zeros
    token_type_ids1 = token_type_ids1 + [0] * (max_length - len(token_type_ids1))  # Pad with zeros
    token_type_ids2 = token_type_ids2 + [0] * (max_length - len(token_type_ids2))  # Pad with zeros


    input_seq1 = torch.tensor([input_ids1])  # Create tensor from token IDs
    input_seq2 = torch.tensor([input_ids2])  # Create tensor from token IDs
    # --- End of Tokenization and Input Preparation ---

    # Create a Transformer encoder
    sample_encoder = Encoder(num_layers=16, d_model=768, num_heads=16,
                             dff=3072, input_vocab_size=102025,  # Updated vocab size
                             maximum_position_encoding=10000,rate=0.3)

    # Pass sentence embeddings through the encoder
    # out1 = sample_encoder(input_seq1, training=False, mask=None)
    # out2 = sample_encoder(input_seq2, training=False, mask=None)
    out1 = sample_encoder(input_seq1, token_type_ids=torch.tensor([token_type_ids1]), training=False, mask=None)
    out2 = sample_encoder(input_seq2, token_type_ids=torch.tensor([token_type_ids2]), training=False, mask=None)

    # print(out1)
    # print(out2)

    # Use the output of the encoder for similarity calculation
    similarity = torch.nn.functional.cosine_similarity(out1.mean(dim=1), out2.mean(dim=1), dim=0)
    # distance = torch.dist(out1.mean(dim=1), out2.mean(dim=1))
    # similarity = 1 / (1 + distance)  # Convert distance to similarity
    # print(similarity)
    # Calculate the mean similarity across the embedding dimension
    mean_similarity = similarity.mean()

    return mean_similarity.item()
    # return similarity.item()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

In [ ]:
pip install --upgrade pandas

In [ ]:
# Define a custom dataset class
class TextSimilarityDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text1, text2, label = self.data[idx]
        return text1, text2, label

# Fine-tuning function
def fine_tune_model(model, dataset, epochs=3, batch_size=8, learning_rate=2e-5):
    """
    Fine-tunes the model on the provided dataset.

    Args:
        model: The model to fine-tune.
        dataset: The dataset for fine-tuning.
        epochs: The number of training epochs.
        batch_size: The batch size for training.
        learning_rate: The learning rate for the optimizer.
    """

    # Split the dataset into training and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Create data loaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

    # Define loss function and optimizer
    loss_fn = nn.MSELoss()  # Use Mean Squared Error loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        for text1, text2, labels in train_dataloader:
            optimizer.zero_grad()  # Clear gradients

            # Calculate similarity scores
            predictions = []
            for t1, t2 in zip(text1, text2):
                try:
                    prediction = calculate_similarity(t1, t2)
                    predictions.append(prediction)
                except Exception as e:
                    print(f"Error calculating similarity for texts: '{t1}', '{t2}'")
                    print(f"Error message: {e}")

            # Convert predictions to tensor after the loop
            predictions = torch.tensor(predictions)

            # Ensure labels have gradients enabled
            labels = torch.tensor(labels, requires_grad=True, dtype=torch.float32)  # Ensure correct type

            # Calculate loss
            loss = loss_fn(predictions, labels)

            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update model parameters

        # Validation
        model.eval()  # Set the model to evaluation mode
        val_predictions = []
        val_labels = []
        with torch.no_grad():  # Disable gradient calculation
            for text1, text2, labels in val_dataloader:
                # Calculate similarity scores (handling potential errors)
                predictions = []
                for t1, t2 in zip(text1, text2):
                    try:
                        prediction = calculate_similarity(t1, t2)
                        predictions.append(prediction)
                    except Exception as e:
                        print(f"Error calculating similarity for texts: '{t1}', '{t2}'")
                        print(f"Error message: {e}")

                # Convert predictions to tensor after the loop
                predictions = torch.tensor(predictions)  # Convert to tensor here

                # Now you can check for NaN values in the predictions tensor
                if torch.isnan(predictions).any():
                    print("NaN values found in predictions!")
                    print(predictions)

                val_predictions.extend(predictions.tolist())
                val_labels.extend(labels.tolist())

                # Convert predictions to tensor after the loop
                predictions = torch.tensor(predictions)

                val_predictions.extend(predictions.tolist())
                val_labels.extend(labels.tolist())
        if torch.isnan(torch.tensor(val_labels)).any():
            print("NaN values found in labels!")
            print(val_labels)
        # Calculate validation MSE
        val_mse = mean_squared_error(val_labels, val_predictions)
        print(f"Epoch {epoch + 1}, Validation MSE: {val_mse}")

# Load the dataset with explicit handling of quoted fields
df = pd.read_csv('expanded_bangla_similarity_dataset_fixed.csv', sep=',', names=["Text 1", "Text 2", "Similarity Score"], header=0)
# data['Similarity Score'] = data['Similarity Score'].fillna(0.5)

# Convert the data to the desired format (ensuring correct data types)
dataset = [(row['Text 1'], row['Text 2'], float(row['Similarity Score'])) for _, row in df.iterrows()]

# Create the dataset object
similarity_dataset = TextSimilarityDataset(dataset, tokenizer)

# Fine-tune the model
fine_tune_model(model, similarity_dataset)

<ipython-input-6-da5781ffcef5>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, requires_grad=True, dtype=torch.float32)  # Ensure correct type
<ipython-input-6-da5781ffcef5>:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(predictions)


Epoch 1, Validation MSE: 0.13728809009963674


<ipython-input-6-da5781ffcef5>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, requires_grad=True, dtype=torch.float32)  # Ensure correct type
<ipython-input-6-da5781ffcef5>:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(predictions)


Epoch 2, Validation MSE: 0.140674499970779


<ipython-input-6-da5781ffcef5>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, requires_grad=True, dtype=torch.float32)  # Ensure correct type
<ipython-input-6-da5781ffcef5>:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(predictions)


Epoch 3, Validation MSE: 0.14003797966567483


In [ ]:
text1 = "বই পড়া একটি ভালো অভ্যাস। বই পড়লে জ্ঞান বৃদ্ধি পায়। বই পড়লে মন ভালো থাকে। বই পড়লে আমরা নতুন নতুন জিনিস শিখতে পারি। বই পড়লে আমাদের চিন্তাশক্তি বৃদ্ধি পায়। বই পড়লে আমরা বিশ্ব সম্পর্কে জানতে পারি। বই পড়লে আমরা অন্যদের সাথে ভালোভাবে যোগাযোগ করতে পারি। বই পড়লে আমরা নিজেদেরকে আরও ভালোভাবে বুঝতে পারি।"
text2 = "বই পড়া  আমাদের  জ্ঞানের  জগৎ  কে  বিস্তৃত  করে।  বই  আমাদের  নতুন  নতুন  চিন্তার  জন্ম  দেয়।  বই  আমাদের  মন  কে  উন্মুক্ত  করে  দেয়।  বই  আমাদের  সৃজনশীলতাকে  উদ্দীপিত  করে।  বই  আমাদের  সমালোচনামূলক  চিন্তাভাবনা  করতে  শেখায়।  বই  আমাদের  সমাজ  ও  বিশ্ব  সম্পর্কে  সচেতন  করে  তোলে।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.8886659741401672
Similarity Score: 0.71875


In [ ]:
text1 = "বাংলাদেশের  সংস্কৃতি  অনেক  সমৃদ্ধ  এবং  বৈচিত্র্যময়।  এ  দেশের  মানুষ  বিভিন্ন  ধর্ম,  বর্ণ  ও  গোষ্ঠীর।  তাদের  মধ্যে  রয়েছে  মুসলিম,  হিন্দু,  বৌদ্ধ,  খ্রিস্টান  ও  উপজাতীয়  গোষ্ঠী।  বাংলাদেশের  সংস্কৃতিতে  রয়েছে  বিভিন্ন  উৎসব,  পার্বণ,  খাবার,  পোশাক,  সঙ্গীত,  নৃত্য  ও  সাহিত্য।  বাংলাদেশের  মানুষ  খুব  ই  আতিথেয়ী  এবং  সামাজিক।  তারা  বিভিন্ন  সামাজিক  অনুষ্ঠান  ও  উৎসবে  অংশগ্রহণ  করে।  বাংলাদেশের  সংস্কৃতি  একটি  গুরুত্বপূর্ণ  অংশ  এ  দেশের  জাতীয়  ঐক্য  ও  ঐতিহ্যের।"
text2 = "ভারতের  সংস্কৃতি  ও  অনেক  সমৃদ্ধ  এবং  বৈচিত্র্যময়।  এ  দেশের  মানুষ  ও  বিভিন্ন  ধর্ম,  বর্ণ  ও  গোষ্ঠীর।  তাদের  মধ্যে  রয়েছে  হিন্দু,  মুসলিম,  শিখ,  খ্রিস্টান,  জৈন  ও  অন্যান্য  ধর্মের  মানুষ।  ভারতের  সংস্কৃতিতে  রয়েছে  বিভিন্ন  উৎসব,  পার্বণ,  খাবার,  পোশাক,  সঙ্গীত,  নৃত্য  ও  সাহিত্য।  ভারতের  মানুষ  ও  খুব  ই  আতিথেয়ী  এবং  সামাজিক।  তারা  বিভিন্ন  সামাজিক  অনুষ্ঠান  ও  উৎসবে  অংশগ্রহণ  করে।  ভারতের  সংস্কৃতি  একটি  গুরুত্বপূর্ণ  অংশ  এ  দেশের  জাতীয়  ঐক্য  ও  ঐতিহ্যের।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.9373860359191895
Similarity Score: 0.8098958134651184


In [ ]:
text1 = "শিক্ষা  মানুষের  মৌলিক  অধিকার।  শিক্ষা  মানুষকে  সচেতন  নাগরিক  হিসেবে  গড়ে  তোলে।  শিক্ষিত  মানুষ  সমাজের  উন্নয়নে  অবদান  রাখতে  পারে।  শিক্ষা  মানুষকে  আত্মনির্ভরশীল  হতে  সাহায্য  করে।  শিক্ষার  মাধ্যমে  মানুষ  জ্ঞান  অর্জন  করে  এবং  সেই  জ্ঞান  ব্যবহার  করে  সমাজের  উন্নয়নে  কাজ  করে।  শিক্ষা  মানুষকে  সুন্দর  ভাবে  বেঁচে  থাকতে  শেখায়।"
text2 = "স্বাস্থ্য  মানুষের  সবচেয়ে  বড়  সম্পদ।  সুস্থ  শরীরে  সুস্থ  মন  বাস  করে।  সুস্থ  মানুষ  কাজ  করতে  পারে  এবং  সমাজের  উন্নয়নে  অংশগ্রহণ  করতে  পারে।  স্বাস্থ্যের  জন্য  পুষ্টিকর  খাবার  খাওয়া  এবং  নিয়মিত  ব্যায়াম  করা  প্রয়োজন।  স্বাস্থ্য  ভালো  থাকলে  মানুষ  দীর্ঘায়ু  লাভ  করে  এবং  সুখী  জীবন  যাপন  করতে  পারে।  স্বাস্থ্য  ভালো  থাকলে  মানুষ  সমাজের  জন্য  কাজ  করতে  পারে।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.8490257263183594
Similarity Score: 0.7213541865348816


In [ ]:
text1 = "পরিবেশ  দূষণ  একটি  গুরুতর  সমস্যা।  পরিবেশ  দূষণের  ফলে  মানুষের  স্বাস্থ্যের  ক্ষতি  হয়।  পরিবেশ  দূষণ  রোধ  করার  জন্য  সবাইকে  সচেতন  হতে  হবে।  পরিবেশ  संरक्षण  করার  জন্য  বৃক্ষরোপণ  করা  উচিত।  পরিবেশ  দূষণ  রোধে  আইন  প্রণয়ন  এবং  তা  বাস্তবায়ন  করা  প্রয়োজন।  পরিবেশ  দূষণ  রোধে  সরকার  বিভিন্ন  পদক্ষেপ  গ্রহণ  করেছে।  পরিবেশ  দূষণ  রোধে  গণসচেতনতা  বৃদ্ধি  করা  প্রয়োজন।  পরিবেশ  দূষণ  রোধে  শিল্প  কারখানাগুলোকে  বর্জ্য  পরিশোধন  করতে  হবে।"
text2 = "জলবায়ু  পরিবর্তন  একটি  বৈশ্বিক  সমস্যা।  জলবায়ু  পরিবর্তনের  ফলে  বিশ্বের  তাপমাত্রা  বৃদ্ধি  পাচ্ছে।  বরফ  গলছে।  সমুদ্রের  পানি  বৃদ্ধি  পাচ্ছে।  এই  জন্য  অনেক  দেশ  জলমগ্ন  হওয়ার  ঝুঁকিতে  আছে।  জলবায়ু  পরিবর্তন  রোধ  করার  জন্য  সবাইকে  এগিয়ে  আসতে  হবে।  জলবায়ু  পরিবর্তন  রোধে  বিশ্ব  নেতারা  বিভিন্ন  পদক্ষেপ  গ্রহণ  করেছেন।  জলবায়ু  পরিবর্তন  রোধে  গণসচেতনতা  বৃদ্ধি  করা  প্রয়োজন।  জলবায়ু  পরিবর্তন  রোধে  কার্বন  নিঃসরণ  কমাতে  হবে।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.8492242097854614
Similarity Score: 0.7526041865348816


In [ ]:
text1 = "খরা  একটি  প্রাকৃতিক  দুর্যোগ  যা  কোন  অঞ্চলে  দীর্ঘ  সময়  ধরে  বৃষ্টিপাতের  অভাবের  কারণে  ঘটে।  খরার  ফলে  মাটি  শুষ্ক  হয়ে  যায়,  ফসল  নষ্ট  হয়  এবং  পানির  সংকট  দেখা  দেয়।  খরা  মানুষ  ও  প্রাণীদের  জীবনযাত্রায়  বিরূপ  প্রভাব  ফেলে।  খরা  অঞ্চলে  জীববৈচিত্র্য  হ্রাস  পায়  এবং  পরিবেশের  ওপর  নেতিবাচক  প্রভাব  ফেলে।"
text2 = "বন্যা  একটি  প্রাকৃতিক  দুর্যোগ  যা  অতিরিক্ত  বৃষ্টিপাত  বা  নদীর  পানি  বৃদ্ধির  কারণে  ঘটে।  বন্যার  ফলে  বাড়িঘর  ডুবে  যায়,  ফসলের  ক্ষতি  হয়  এবং  মানুষ  ও  প্রাণী  জীবনের  ঝুঁকিতে  পড়ে।  বন্যা  পরিবেশের  ওপর  বিরূপ  প্রভাব  ফেলে।  বন্যার  ফলে  পানিবাহিত  রোগ  বৃদ্ধি  পায়  এবং  অর্থনৈতিক  ক্ষতি  হয়।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.9146221280097961
Similarity Score: 0.7708333134651184


In [ ]:
text1 = "বাংলাদেশ  একটি  নদীমাতৃক  দেশ।  এখানে  অসংখ্য  নদী  প্রবাহিত  হয়,  যা  দেশের  অর্থনীতি  এবং  সংস্কৃতিতে  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।  নদীগুলো  কৃষি,  মৎস্য  চাষ,  যোগাযোগ  এবং  পরিবহনের  জন্য  অত্যাবশ্যক।  নদী  বাংলাদেশের  জীবনধারার  একটি  অবিচ্ছেদ্য  অংশ।  নদী  বাংলাদেশের  প্রাকৃতিক  সৌন্দর্য  বৃদ্ধি  করে  এবং  জীববৈচিত্র্য  সংরক্ষণে  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।"
text2 = "বাংলাদেশের  নদ-নদী  দেশের  অর্থনীতিতে  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।  নদী  থেকে  মাছ  আহরণ  এবং  কৃষিকাজের  জন্য  পানি  সরবরাহ  করা  হয়।  নদীপথ  যোগাযোগ  ব্যবস্থার  একটি  গুরুত্বপূর্ণ  অংশ।  নদী  বাংলাদেশের  প্রাকৃতিক  সৌন্দর্য  বৃদ্ধি  করে।  নদী  বাংলাদেশের  সংস্কৃতি  ও  ঐতিহ্যের  একটি  অবিচ্ছেদ্য  অংশ।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.9372631907463074
Similarity Score: 0.6901041865348816


In [ ]:
text1 = "বাংলাদেশের  জলবায়ু  উষ্ণ  ও  আর্দ্র।  এখানে  ছয়টি  ঋতু  আছে:  গ্রীষ্ম,  বর্ষা,  শরৎ,  হেমন্ত,  শীত  ও  বসন্ত।  প্রতিটি  ঋতুর  আলাদা  আলাদা  সৌন্দর্য  আছে।  বাংলাদেশের  জলবায়ু  কৃষির  জন্য  উপযোগী।  বাংলাদেশ  একটি  নদীমাতৃক  দেশ  হওয়ায়  এখানে  প্রচুর  বৃষ্টিপাত  হয়।  বর্ষাকালে  প্রায়ই  বন্যা  হয়।  গ্রীষ্মকালে  তাপমাত্রা  বেশ  বেড়ে  যায়।  শীতকালে  তাপমাত্রা  কমে  যায়  এবং  আবহাওয়া  শুষ্ক  থাকে।"
text2 = "জলবায়ু  পরিবর্তনের  ফলে  বাংলাদেশের  জলবায়ুতে  বিভিন্ন  পরিবর্তন  দেখা  দেচ্ছে।  তাপমাত্রা  বৃদ্ধি  পাচ্ছে।  বৃষ্টিপাতের  পরিমাণ  ও  ধরণ  পরিবর্তিত  হচ্ছে।  সমুদ্রপৃষ্ঠের  উচ্চতা  বৃদ্ধি  পাচ্ছে।  এসব  পরিবর্তনের  ফলে  বাংলাদেশে  প্রাকৃতিক  দুর্যোগ  বৃদ্ধি  পাচ্ছে।  জলবায়ু  পরিবর্তনের  ফলে  বাংলাদেশের  কৃষি,  মৎস্য  এবং  অন্যান্য  খাত  ক্ষতিগ্রস্ত  হচ্ছে।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

Similarity Score: 0.7213541865348816


In [ ]:
text1 = "সুতরাং একদম ঘাবড়াবেন না।"
text2 = "তাই একদম ভয় পেও না।"
'''from there the process is started its called calculate_similarity with two input text'''
similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.7399076819419861
Similarity Score: 0.4609375


In [ ]:
text1 = "ঘনবসতিপূর্ণ দেশ যেখানে প্রাকৃতিক সম্পদের প্রাচুর্য বিদ্যমান।"
text2 = "জনবহুল  রাষ্ট্র  যেখানে  প্রচুর  প্রাকৃতিক  সম্পদ  আছে।"
similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.6565853357315063
Similarity Score: 0.6041666865348816


In [ ]:
text1 = "বাংলাদেশ একটি ঘনবসতিপূর্ণ দেশ যেখানে প্রাকৃতিক সম্পদের প্রাচুর্য বিদ্যমান। এই দেশের অর্থনীতি মূলত কৃষি  নির্ভর, যদিও  গার্মেন্টস  শিল্প  এবং  অন্যান্য  খাত  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।  বাংলাদেশের  মানুষ  কঠোর  পরিশ্রমী  এবং  তারা  দেশের  অগ্রগতির  জন্য  নিষ্ঠার  সাথে  কাজ  করে।  এ  দেশের  সংস্কৃতি  অনেক  সমৃদ্ধ  এবং  এখানে  বিভিন্ন  ধর্ম  ও  বর্ণের  মানুষ  একসাথে  বাস  করে।  বাংলাদেশ  প্রাকৃতিক  সৌন্দর্যে  ভরা  একটি  দেশ  যেখানে  অনেক  নদ-নদী,  হাওর-বাঁওড়  এবং  সবুজ  শ্যামল  মাঠ  রয়েছে।  এ  দেশের  মানুষ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।"
text2 = "বাংলাদেশ  একটি  জনবহুল  রাষ্ট্র  যেখানে  প্রচুর  প্রাকৃতিক  সম্পদ  আছে।  এই  রাষ্ট্রের  অর্থনীতি  প্রধানত  কৃষি  কেন্দ্রিক,  যদিও  পোশাক  শিল্প  এবং  অন্যান্য  ক্ষেত্র  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে  থাকে।  বাংলাদেশের  নাগরিকরা  অত্যন্ত  পরিশ্রমী  এবং  তারা  রাষ্ট্রের  উন্নয়নের  জন্য  আন্তরিকতার  সাথে  কর্ম  করে।  এই  রাষ্ট্রের  সংস্কৃতি  অত্যন্ত  সমৃদ্ধ  এবং  এখানে  নানা  ধর্ম  ও  বর্ণের  লোক  একত্রে  বসবাস  করে।  বাংলাদেশ  প্রাকৃতিক  রূপ-লাবণ্যে  পরিপূর্ণ  একটি  রাষ্ট্র  যেখানে  অনেক  নদী,  জলাভূমি  এবং  সবুজ  ক্ষেত  আছে।  এই  রাষ্ট্রের  জনগণ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.8864535093307495
Similarity Score: 0.7734375


In [ ]:
text1 = "বাংলাদেশ,  একটি  নবীন  প্রজাতন্ত্র,  ১৯৭১  সালে  একটি  রক্তক্ষয়ী  স্বাধীনতা  যুদ্ধের  মাধ্যমে  জন্ম  লাভ  করে।  এই  ক্ষুদ্র  দেশটি  ঘনবসতিপূর্ণ  এবং  প্রাকৃতিক  সম্পদের  অভাব  থাকা  সত্ত্বেও  অর্থনৈতিক  উন্নয়নের  পথে  অগ্রসর  হচ্ছে।  বাংলাদেশের  অর্থনীতি  মূলত  কৃষি  নির্ভর,  যদিও  গার্মেন্টস  শিল্প  এবং  রেমিট্যান্স  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।  বাংলাদেশের  মানুষ  কঠোর  পরিশ্রমী  এবং  তারা  দেশের  অগ্রগতির  জন্য  নিষ্ঠার  সাথে  কাজ  করে।  এ  দেশের  সংস্কৃতি  অনেক  সমৃদ্ধ  এবং  এখানে  বিভিন্ন  ধর্ম  ও  বর্ণের  মানুষ  একসাথে  বাস  করে।  বাংলাদেশ  প্রাকৃতিক  সৌন্দর্যে  ভরা  একটি  দেশ  যেখানে  অনেক  নদ-নদী,  হাওর-বাঁওড়  এবং  সবুজ  শ্যামল  মাঠ  রয়েছে।  এ  দেশের  মানুষ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।  বাংলাদেশ  একটি  গণতান্ত্রিক  দেশ  যেখানে  জনগণ  তাদের  নির্বাচিত  প্রতিনিধিদের  মাধ্যমে  দেশ  পরিচালনা  করে।  বাংলাদেশ  বিভিন্ন  আন্তর্জাতিক  সংস্থার  সদস্য  এবং  বিশ্ব  শান্তি  ও  উন্নয়নে  অবদান  রাখছে।  বাংলাদেশ  একটি  উন্নয়নশীল  দেশ  এবং  এ  দেশের  মানুষ  শিক্ষা,  স্বাস্থ্য  ও  অন্যান্য  ক্ষেত্রে  উন্নতি  লাভ  করছে।  বাংলাদেশ  একটি  সম্ভাবনাময়  দেশ  এবং  এ  দেশের  ভবিষ্যৎ  উজ্জ্বল।বাংলাদেশ একটি ঘনবসতিপূর্ণ দেশ যেখানে প্রাকৃতিক সম্পদের প্রাচুর্য বিদ্যমান। এই দেশের অর্থনীতি মূলত কৃষি  নির্ভর, যদিও  গার্মেন্টস  শিল্প  এবং  অন্যান্য  খাত  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে।  বাংলাদেশের  মানুষ  কঠোর  পরিশ্রমী  এবং  তারা  দেশের  অগ্রগতির  জন্য  নিষ্ঠার  সাথে  কাজ  করে।  এ  দেশের  সংস্কৃতি  অনেক  সমৃদ্ধ  এবং  এখানে  বিভিন্ন  ধর্ম  ও  বর্ণের  মানুষ  একসাথে  বাস  করে।  বাংলাদেশ  প্রাকৃতিক  সৌন্দর্যে  ভরা  একটি  দেশ  যেখানে  অনেক  নদ-নদী,  হাওর-বাঁওড়  এবং  সবুজ  শ্যামল  মাঠ  রয়েছে।  এ  দেশের  মানুষ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।"
text2 = "স্বাধীনতা  যুদ্ধের  মাধ্যমে  ১৯৭১  সালে  প্রতিষ্ঠিত  বাংলাদেশ  একটি  তরুণ  প্রজাতন্ত্র।  জনবহুল  এবং  প্রাকৃতিক  সম্পদের  দিক  থেকে  অপেক্ষাকৃত  দরিদ্র  হওয়া  সত্ত্বেও  এই  ছোট্ট  রাষ্ট্র  অর্থনৈতিক  অগ্রগতির  দিকে  এগিয়ে  চলছে।  বাংলাদেশের  অর্থনীতি  প্রধানত  কৃষি  কেন্দ্রিক,  যদিও  পোশাক  শিল্প  এবং  প্রবাসী  আয়  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে  থাকে।  বাংলাদেশের  নাগরিকরা  অত্যন্ত  পরিশ্রমী  এবং  তারা  রাষ্ট্রের  উন্নয়নের  জন্য  আন্তরিকতার  সাথে  কর্ম  করে।  এই  রাষ্ট্রের  সংস্কৃতি  অত্যন্ত  সমৃদ্ধ  এবং  এখানে  নানা  ধর্ম  ও  বর্ণের  লোক  একত্রে  বসবাস  করে।  বাংলাদেশ  প্রাকৃতিক  রূপ-লাবণ্যে  পরিপূর্ণ  একটি  রাষ্ট্র  যেখানে  অনেক  নদী,  জলাভূমি  এবং  সবুজ  ক্ষেত  আছে।  এই  রাষ্ট্রের  জনগণ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।  বাংলাদেশ  একটি  গণতান্ত্রিক  রাষ্ট্র  যেখানে  জনগণ  তাদের  নির্বাচিত  প্রতিনিধিদের  মাধ্যমে  রাষ্ট্র  পরিচালনা  করে।  বাংলাদেশ  বিভিন্ন  আন্তর্জাতিক  সংগঠনের  সদস্য  এবং  বিশ্ব  শান্তি  ও  উন্নয়নে  অবদান  রাখছে।  বাংলাদেশ  একটি  উন্নয়নশীল  রাষ্ট্র  এবং  এ  রাষ্ট্রের  জনগণ  শিক্ষা,  স্বাস্থ্য  ও  অন্যান্য  ক্ষেত্রে  উন্নতি  লাভ  করছে।  বাংলাদেশ  একটি  সম্ভাবনাময়  রাষ্ট্র  এবং  এ  রাষ্ট্রের  ভবিষ্যৎ  উজ্জ্বল। বাংলাদেশ  একটি  জনবহুল  রাষ্ট্র  যেখানে  প্রচুর  প্রাকৃতিক  সম্পদ  আছে।  এই  রাষ্ট্রের  অর্থনীতি  প্রধানত  কৃষি  কেন্দ্রিক,  যদিও  পোশাক  শিল্প  এবং  অন্যান্য  ক্ষেত্র  গুরুত্বপূর্ণ  ভূমিকা  পালন  করে  থাকে।  বাংলাদেশের  নাগরিকরা  অত্যন্ত  পরিশ্রমী  এবং  তারা  রাষ্ট্রের  উন্নয়নের  জন্য  আন্তরিকতার  সাথে  কর্ম  করে।  এই  রাষ্ট্রের  সংস্কৃতি  অত্যন্ত  সমৃদ্ধ  এবং  এখানে  নানা  ধর্ম  ও  বর্ণের  লোক  একত্রে  বসবাস  করে।  বাংলাদেশ  প্রাকৃতিক  রূপ-লাবণ্যে  পরিপূর্ণ  একটি  রাষ্ট্র  যেখানে  অনেক  নদী,  জলাভূমি  এবং  সবুজ  ক্ষেত  আছে।  এই  রাষ্ট্রের  জনগণ  তাদের  ঐতিহ্য  ও  সংস্কৃতি  নিয়ে  গর্বিত।"

similarity_score = calculate_similarity(text1, text2)
print(f"Similarity Score: {similarity_score}")

0.9328982830047607
Similarity Score: 0.8255208134651184


In [ ]:
df = pd.read_csv('expanded_bangla_similarity_dataset_fixed.csv', sep=',', names=["Text 1", "Text 2", "Similarity Score"], header=0)
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

FileNotFoundError: [Errno 2] No such file or directory: 'expanded_bangla_similarity_dataset_fixed.csv'

In [ ]:
print(f'\nShape of the dataframe: {df.shape}')


Shape of the dataframe: (135, 3)


In [ ]:
print("\nText 1 column:")
print(df['Text 1'].tolist())


Text 1 column:
['Text 1', 'বাংলাদেশের অর্থনীতি ক্রমশ শক্তিশালী হচ্ছে। গার্মেন্টস শিল্প বৈদেশিক মুদ্রা অর্জনের একটি প্রধান উৎস। কৃষি এবং শিল্প বাংলাদেশের অর্থনীতির ভিত্তি।', 'স্বাস্থ্য সুরক্ষার জন্য নিয়মিত ব্যায়াম করা প্রয়োজন। সুস্থ থাকার জন্য সুষম খাবার খাওয়া গুরুত্বপূর্ণ। পর্যাপ্ত ঘুম এবং বিশ্রাম স্বাস্থ্য বজায় রাখার জন্য আবশ্যক।', 'প্রযুক্তির উন্নতির ফলে মানুষের জীবন অনেক সহজ হয়েছে। আধুনিক প্রযুক্তি দ্রুত যোগাযোগের সুযোগ দিয়েছে। প্রযুক্তি শিক্ষা এবং গবেষণার ক্ষেত্রেও অনেক পরিবর্তন এনেছে।', 'বাংলাদেশের প্রধান খাদ্যশস্য হলো ধান। দেশের বেশিরভাগ মানুষ কৃষিকাজের সাথে জড়িত। ধান চাষ বাংলাদেশের অর্থনীতিতে গুরুত্বপূর্ণ ভূমিকা পালন করে।', 'বাংলাদেশের জাতীয় খেলা হলো কাবাডি। ক্রিকেট বাংলাদেশের মানুষের কাছে অত্যন্ত জনপ্রিয়। খেলাধুলা মানসিক এবং শারীরিক বিকাশে সাহায্য করে।', 'বাংলাদেশ একটি নদীমাতৃক দেশ। এখানে অসংখ্য নদী প্রবাহিত হয়,যা দেশের অর্থনীতি এবং সংস্কৃতিতে গুরুত্বপূর্ণ ভূমিকা পালন করে। নদীগুলো কৃষি,মৎস্য চাষ,যোগাযোগ এবং পরিবহনের জন্য অত্যাবশ্যক।', 'আধুনিক প্রযুক্তির উন্নয়ন আমাদ

In [ ]:
print("\nText 2 column:")
print(df['Text 2'].tolist())


Text 2 column:
['Text 2', 'রপ্তানি খাতে গার্মেন্টস শিল্পের অবদান অত্যন্ত গুরুত্বপূর্ণ। দেশের অর্থনীতি কৃষি,শিল্প এবং সেবার উপর নির্ভরশীল।', 'সুস্থ থাকার জন্য সঠিক জীবনযাপন পদ্ধতি অনুসরণ করতে হবে। নিয়মিত স্বাস্থ্য পরীক্ষা আমাদের ভালো থাকার সহায়ক।', 'ইন্টারনেট প্রযুক্তি আমাদের দৈনন্দিন কাজকে সহজ করেছে। প্রযুক্তি আমাদের শিক্ষার পদ্ধতিতে বৈপ্লবিক পরিবর্তন এনেছে।', 'ধান থেকে চাল তৈরি হয় যা বাংলাদেশের মানুষের প্রধান খাদ্য। ধান চাষ কৃষি নির্ভর বাংলাদেশের অর্থনীতির মূল চালিকা।', 'বাংলাদেশে ক্রিকেট একটি আবেগের বিষয়। ফুটবল এবং কাবাডি দেশের ঐতিহ্যবাহী খেলার অংশ। খেলাধুলা আমাদের স্বাস্থ্য এবং মনোভাব উন্নত করে।', 'বাংলাদেশের অর্থনীতি মূলত কৃষি-নির্ভর। দেশের অধিকাংশ  মানুষ  কৃষি  কাজের  সাথে  জড়িত।  ধান,পাট,চা  ইত্যাদি  প্রধান  কৃষি  পণ্য।  কৃষি  ক্ষেত্রে  উন্নয়ন  বাংলাদেশের  অর্থনৈতিক  উন্নয়নের  জন্য  গুরুত্বপূর্ণ।', 'প্রযুক্তির  ব্যবহার  শিক্ষা  ক্ষেত্রে  বিরাট  ভূমিকা  পালন  করছে।  অনলাইন  শিক্ষা  ব্যবস্থা  এখন  অনেক  জনপ্রিয়।  ছাত্ররা  ঘরে  বসে  ইন্টারনেটের  মাধ্যমে  বিভিন্ন  বিষয়ে  জ্

In [ ]:
print("\nSimilarity Score column:")
print(df['Similarity Score'].tolist())


Similarity Score column:
['Similarity Score', '0.92', '0.96', '1', '1', '0.81', '0.4', '0.6', '0', '0', '0.2', '0.6', '0', '0', '0', '1', '0.2', '0.4', '0.5', '0.7', '0.5', '0.7', '0.6', '0.7', '0.8', '0.6', '0.7', '0.8', '0.6', '0.5', '0.6', '0.7', '0.8', '0.5', '0.6', '0.7', '0.5', '0.6', '0.7', '0.5', '0.6', '0.7', '0.5', '0.6', '0.7', '0.6', '0.5', '0.6', '0.7', '0.8', '0.6', '0.7', '0.5', '0.6', '0.7', '0.8', '0.6', '0.5', '0.7', '0.8', '0.6', '0.5', '0.6', '0.5', '0.4', '0.5', '0.6', '0.4', '0.5', '1', '0.82', '0.85', '1', '1', '0', '0.6', '0.7', '0.6', '0.7', '0.8', '1', '0.45', '0.25', '0.68', '0.53', '0.80', '0.45', '0', '0', '0', '0', '0', '0.4', '0', '0.37', '0.04', '0', '0.62', '0', '0.32', '0', '0', '0', '0.88', '0.77', '0.08', '0', '0', '0.4', '0', '0.77', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0.67', '0', '0', '0', '0', '0', '0.12', '0.19', '0.17', '0.96', '0.96', '0.20']
